In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from functools import reduce
from collections import Iterable

# use for vertify 
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
real_dataset_path = "/home/sokhorn/sokhorn/dataSet/data/Online Retail.csv"
testing_dataset_path = '/home/sokhorn/sokhorn/dataSet/data/sample_data_set.csv'
sample_dataset = pd.read_csv(
    real_dataset_path, sep=',', usecols=[
        'InvoiceNo',
        'StockCode',
        'Quantity'
    ])
sample_dataset

In [ ]:
item_sets = (
    sample_dataset.groupby(['InvoiceNo', 'StockCode', ])['Quantity']
    .sum().unstack().reset_index().fillna(0)
    .set_index("InvoiceNo")
)
item_sets


In [ ]:
en_code = lambda x : 1 if x > 0  else 0
item_sets = item_sets.applymap(en_code)
item_sets

**Generating RC column for each tranctions**

In [ ]:
def rc(item_np):
    np_hash = {}
    for item in item_np:
        key = " ".join(map(str, item))
        if np_hash.get(key):
            np_hash[key] += 1
        else:
            np_hash[key] = 1
    item_rc = []
    for item in np_hash:
        values =  list(map(int,  item.split()))
        values.append(np_hash[item])
        item_rc.append(values)
    return item_rc


In [ ]:
df_rc =  pd.DataFrame(np.array(rc(item_sets.values)))
columns = item_sets.columns
columns.append("RC")
df_rc.set_axis(columns, inplace=True  , axis=1 )
RC = df_rc['RC']
df_rc.drop(['RC'], axis=1, inplace=True)
df_rc

In [ ]:
def support_k_itemst(k_itemst):
    s = 0
    for i in range(len(k_itemst)):
        s += reduce(lambda a, b: a & b, k_itemst[i] & RC[i])
    return s

In [ ]:
def count_ocurence(itemset, Tranctions):
    count = 0
    for i in range(len(Tranctions)):
        if set(itemset).issubset(set(Tranctions[i])):
            count += 1
    return count

In [ ]:
def join_two_itemsets(it1, it2, order):
    it1.sort(key=lambda x : order.index(x))
    it2.sort(key=lambda x : order.index(x))

    for i in range(len(it1) - 1): # check befor the last one 
       if it1[i] != it2 [i] :
           return []
    
    if order.index(it1[-1]) < order.index(it2[-1]) :
        return [it1] + [it2[-1]]

    return []


In [ ]:
def mergeKItemIntoOne(ab):
    result = []
    if(len(ab) != 0):
        if(len(ab[0]) == 1):
            return ab
        else:
            for item in ab:
                res = item[0] + item[1].split()
                result.append(res)
        return result


In [ ]:
def get_frequent(itemesets, min_support, prev_discard):
    L = []
    support_count = []
    new_discard = []
    column_items = mergeKItemIntoOne(itemesets)

    k = len(prev_discard)
    for i in range(len(itemesets)):
        discard_before = False
        item = itemesets[i] 
        result = []
        for i_item in item:
            if isinstance(i_item, list):
                for j in i_item:
                    result.append(j)
            else:
                result.append(i_item)

        if k > 0:
            for it in prev_discard[k]:
                if set(it).issubset(set(result)):
                    discard_before = True
                    break
        
        if not discard_before:
            # print(f'item {mergeKItemIntoOne(item)}')
            count = support_k_itemst(df_rc[column_items[i]].values)
            count = 1
            if count >= min_support:
                L.append(result)
                support_count.append(count)
            else:
                new_discard.append(result)
    return L, support_count, new_discard

In [ ]:
def join_set_item(set_of_its, order_column_name):
    C = []
    for i in range(len(set_of_its)):
        for j in range(i + 1, len(set_of_its)):
            it_out = join_two_itemsets(
                set_of_its[i], set_of_its[j], order_column_name)
            if(len(it_out)) > 0:
                C.append(it_out)

    return C


**Generating 1 itemse base on support count of row**

In [ ]:
C = {}
L = {}
k_items = []
Discard = {}
itemset_size = 1
min_support = 1
Discard.update({itemset_size : []})

#### Generating 1 itemset

In [ ]:
df_rc

In [ ]:
# remove each column who support count are less than min_support for 1 itemsets
rc_1_itemset = df_rc.sum(axis=0)
cut_our_cols = rc_1_itemset.loc[lambda x: x < min_support].index
cut_our_cols
df_rc.drop(labels=cut_our_cols, axis=1, inplace=True)
df_rc

In [ ]:
C.update({itemset_size: np.reshape(list(df_rc.columns), (-1, 1))})

In [ ]:
support_count = {}
f, supp, new_discard = get_frequent(
    C[itemset_size], min_support, Discard
)
Discard.update({itemset_size: new_discard})
L.update({itemset_size: f})
support_count.update({itemset_size: supp})

In [ ]:
k = itemset_size + 1
convergence = False
while not convergence:
    C.update({k: join_set_item(L[k - 1], list( df_rc.columns))})
    f, supp, new_discard = get_frequent(C[k], min_support, Discard)
    L.update({k: f})
    Discard.update({k: new_discard})
    support_count.update({k: supp})
    if(len(L[k]) == 0):
        convergence = True
    k += 1

In [ ]:
itemset = apriori(item_sets, min_support=min_support / len(item_sets), use_colnames=True)
itemset